In [3]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.io.wavfile
from scipy.fftpack import fft, fftfreq

# Error-Correcting Codes

## What are Error-Correcting codes and why are they so important?

Error-correcting codes (ECC) are techniques used in digital communications and storage systems to detect and correct errors in data. These errors can occur due to noise, interference, or other issues during transmission or storage. These codes add redundancy to the original information so that errors can be identified and fixed.

At the core, ECC involve mathematical algorithms that apply rules for encoding and decoding data. When data is sent over a network or stored in a device, these codes can identify unintentional changes to the information. By addressing errors such as bits flipping from 1 to 0 or vice versa, ECC help maintain data integrity. The idea is not only to detect but also to correct data to its original state without requiring re-transmission, making them incredibly valuable for efficient and reliable data communication.

## What are the different types of error-correcting codes? Provide real-world examples?

There are two main types of ECC:

- Block codes - they work on fix-size block or symbols of predefined size. Well suited for correcting random errors. Examples of such codes are the Hamming codes, Reed-Solomon, and BSH codes. [1]
- Convolutional codes - process the data bit by bit (bit or string streams of arbitrary length), that generates parity symbols via the sliding application of a boolean polynomial function to the data stream. The sliding application represents the "convolution" of the encoder over the data.[2]


The most important classes of the block error correcting codes are :
- Hamming codes - perfect codes for single errors.
- Reed-Muller codes - multiple-error-correcting codes with a particularly efficient and easily implemented decoder.
- Golay code - the unique perfect code for triple errors.
- BCH codes - strong multiple-error-correcting codes 
with a fast decode.[3]


## What are parity bits and how do we use them?

Let us first begin with this : what is Parity Check? The parity check is a simple error detection technique used to determine whether a binary data set has been altered during transmission or storage. For a parity check, we separate only one single bit, that the sender is responsible for tuning, and the rest bits are free to carry a message. The only job of this single bit is to make sure, that the total number of 1s in this binary data is an even number. For example, if the original binary data is [1101001], here the number of the 1s is 4, which is an even number. So the value of this special bit will be 0 and the transmitted data will be  [11010010] (the orinal data plus the special bit). But if we would like to transfer the data [1111 111], here the number of the 1s is odd, that is why the sender needs to flip that special bit to be 1, in order to make the count even and the transmitted data will be  [1111 1111]. And this special bit is named "parity bit". This is pretty simple, but still very elegant way a change anywhere in the binary data to be reflected in a single bit of information.
Of course, this is a very simple check - if there are two errors in the transmission, the number of the 1s still will be even, so the parity check will not show us, that there is a problem. Also if the parity check shows an error, it could be not one error, but 3 or 7 or 127. 
So parity checks on their own are pretty weak, but by distilling the idea of change across full message down to a single bit, give room for more sophisticated schemes.

## What is the Hamming distance and what is its significance? How is it related to other distance metrics for text / bit sequences?

Now let us define what is a code word - a code word is the (binary)string or the sequence of symbols, that is the output of the encoding process.
The main idea of the error correction is that we want to have code words sufficiently "wide apart" in order to recognize the corrupted positions of the received word [3]

In the information theory, the Hamming distance between any two bit strings with equal length is the number of positions, in which the two strings differ. For example, if we have the strings [101101] and [100110], then the Hamming distance between the two strings is 3.



Also let us clarify the difference between Error Detection and Error Correction. The Error Detection means that the code can detect errors, but cannot say where in the received sequence the errors are. Error correction means we know where the errors are and can correct the bit positions in error. Of course, if a code can correct up to t errors, but the received message contains more than t errors, then the decoder may decode to the wrong information sequence.

A code with minimum Hamming distance d between any two code words can detect up to d-1 error and correct (d-1)/2 errors with nearest neighbor decoding. Thus, to correct one 1 error, the minimum Hamming distance of a code has to be 3. [11]

While the Hamming distance is specific to binary or textual strings with equal length, there are other distance metrics that measure similarities or differences between sequences:

- Levenshtein distance - measures the minimum numbers of single character edits (insertion, deletion, substitution) required to change one string into another. Unlike the Hamming distance, the Levenshtein distance can handle strings of different length:
  Example - The Hamming distance between [10101010] and [01010101] is 8, but the Levenshtein distance is only 2 - deleting the first symbol and adding the same symbol at the end.

- Eucledian distance - it is a measure of the straight-line distance between two points in Euclidean space. In a two-dimensional plane, the Euclidean distance between points $ p(p_1, p_2) $ and $ q(q_1, q_2) $ is calculated using the Pythagorian theorem: $ \sqrt {(p_1 - q_1)^2 + (p_2 - q_2)^2} $. $ Euclidian_distance = \sqrt Hamming_distance $.

## Binary linear codes

We shall use the addition modulo 2 and the multiplications modulo 2 operations over symbols '1' and '0'. These operations turn the binary alphabet into a field $ Z_2 $, because with the two operations it satisfies all the properties - closure, associativity, commucativity, distributivity, identity elements, additive inversion, multiplicative inversion.
 The binary linear error correcting codes are defined over this field  $ Z_2 $. They are called linear, because they can be described by  systems of linear equations with binary coefficients (which practically means without coefficients - a variable is either present with coefficient 1 or missing with coefficient 0) [3] - p.64

Whenever a binary code can be described by a system of homogenous linear equations, then its code words form a linear space. That is :
- a sum a + b of code words a and b (that is the ith element of the sum is $ a_i + b_i$) is a code word.
- a scalar multiple of code word $\lambda a$ of a code word (whose ith element is $\lambda a_i$) is a code word, for $\lambda$ = 0, 1. [3] - p.65

Then it is said, that a binary block code is said to be linear, provided that the sum of arbitrary two code words is a code word.  |


### Parity check matrix

Given a linear code, described by a system of homogenous equations, we denot by _H_ the matrix of coefficients of that system. That is, the  ith row ot that _H_ describes the ith equation (in the sense, that the jth entry is 1 if the jth variable is included, and 0 otherwise). 

A binary matrix _H_ is called parity check matrix of a linear binary code _K_ of length n provided that the code words of _K_ are precisely those binary words of _K_  $x_{1}$, $x_{2}$....$x_{n}$, which fulfil the following: 

$$ H \left[ \begin{array}{c}x_{1} \\x_{2} \\\vdots \\x_{n}\end{array}\right] = \left[ \begin{array}{c}0 \\0 \\\vdots \\0\end{array}\right]$$

## What is a Hamming code? The history behind it.

In computer science and telecommunications the Hamming codes are linear error correcting codes. Hamming codes can detect one-bit and two-bit errors, or correct one-bit error without detection of uncorrected errors.
In mathematical terms, Hamming codes are a class of binary linear code. For every r >=2 there is code word with block length $ n = 2^r - 1 $ and message length $ k = 2^r - r - 1 $. The Hamming codes are with minimal distance of three - that means, that the minimal number of bit changes needed to go from any code word to any other code word is three. 

Richard Hamming, the inventor of Hamming codes, worked at Bell Labs in the late 1940s on the Bell Model V computer, an electromechanical relay-based machine with cycle times in seconds. Input was fed in on punched paper tape, seven-eighths of an inch wide, which had up to six holes per row. During weekdays, when errors in the relays were detected, the machine would stop and flash lights so that the operators could correct the problem. During after-hours periods and on weekends, when there were no operators, the machine simply moved on to the next job.

Hamming worked on weekends, and grew increasingly frustrated with having to restart his programs from scratch due to detected errors. In a taped interview, Hamming said, "And so I said, 'Damn it, if the machine can detect an error, why can't it locate the position of the error and correct it?'" Over the next few years, he worked on the problem of error-correction, developing an increasingly powerful array of algorithms. In 1950, he published what is now known as Hamming code, which remains in use today in applications such as ECC memory[4].


## Number of parity bits required for a given number of data bits

To determine the number of parity bits required for a given number of data bits in a Hamming code, we need to ensure that the code can detect and correct single-bit errors. The process involves using the parity bits to cover all the data bits and the parity bits themselves. Here's the step-by-step derivation of the formula:
let us denote :
- d is the number of the data bits
- p is the number of the parity bits
- n is the total number of bits (n = d + p)
For hHamming code, each possible bit possition in the data bits must me uniquely identifiable by the combination of the parity bits. The parity bits can form $ 2 ** p $combinations, and hence, we can conclude that these combinations should be greater that the total number of bits:

$$ 2^p >= n $$

Looks reasonable, correct? But we forget something - as it can happen, that there are errors in the data transmission, and we would like to be able to identify them uniquely, there is also the opportunity there not to be error at all, and one of the parity bits combinations should be used to indicate no error at all. That is why the condition should be:
$$ 2^p >= n + 1$$

And as 
$$ n = d + p $$
the formula for required parity bits for a given number of data bits in a Hamming code finally looks like this :
$$ 2^p >= d + p + 1 $$

Let us check how many parity bits for 10 data bits (d = 10). So we need to find such p, that 
$$ 2^p >= 10 + p + 1 $$
As $2^3$ is  8, obviously we shall need a bigger number. Let is try with $p = 4$. 
$$ 2^4 = 16 >= 10 + 4 + 1 $$
This is true, so obviously for 10 data bits 4 parity bits are sufficient.

So generally the method for finding the needed parity bits is :

Find the smallest p, that satisfies $ 2^p >= d + p + 1 $

### General algorithm for encoding data using the Hamming codes

The first step, that has to be done, is, of course, to determine the parity bits, that we need for our data.

The parity bits are placed at positions, which are powers of 2, starting from 0 - 1($2^0$), 2($2^1$), 4($2^2$),  8($2^3$) and so on. The data bits are placed in the remaining positions. 
But why it is chosen the parity bits to be at these places? The main idea is to choose the error-correcting bits such that the index-XOR (the XOR of all the bit positions containing a 1) is 0.(Source : https://en.wikipedia.org/wiki/Hamming_code). This choice allows each parity bit to cover a specific set of bit positions in a systematic way, ensuring that each data bit is checked by multiple parity bits in a non-overlapping manner. Each parity bit at position $2^k$ covers the overs all bit positions that have the k-th bit in their binary representation set to 1.
Each data bit is included in a unique set of 2 or more parity bits, as determined by the binary form of its bit position.

Thus :
- Parity bit at position 1 covers all bit positions which have the least significant bit set: bit 1 (the parity bit itself), 3, 5, 7, 9, etc.
- Parity bit at position 2 covers all bit positions which have the second least significant bit set: bits 2-3, 6-7, 10-11, etc.
- Parity bit at position 4 covers all bit positions which have the third least significant bit set: bits 4-7, 12-15, 20-23, etc.
- Parity bit at position 8 covers all bit positions which have the fourth least significant bit set: bits 8-15, 24-31, 40-47, etc.

### Encoding example

Let we want to transmit this data - 01001001. This is the binary representation of 73, The ASCII code for the letter "I".

As we want to transmit 8 bits, the first thing, that we shall have to do, is to find the smallest p, for which $ 2^p >= d + p + 1 $. Obviously this is 4, as $ 2^4 = 16 >= 8 + 4 + 1 $. So, we need 4 parity bits.
As described in the algorithm above, the parity bits will be at postions, that are power of 2. Thus, the total data, that will be transmitted, will look like this :

P1P20P4100P81001, and with "Px" we mark the positions of the parity bits.

Let's now calcuate the values of the parity bits:

P1, which covers the bit positions, which have the least significant bit set - 3, 5, 7, 9, 11. We calculate it, with the XOR(⊕) operation between the bits, which this parity bit covers. 

So we get P1 = 0 ⊕ 1 ⊕ 0 ⊕ 1 ⊕ 0 = 0

Respectfully, P2 we shall calculate from the bits, which have the second least significant bit set - positions 3, 6, 7, 10, 11. 

So we get P2 = 0 ⊕ 0 ⊕ 0 ⊕ 0 ⊕ 0 = 0

P4 covers the positions  5, 6, 7, 12:

So we get P4 = 1 ⊕ 0 ⊕ 0 ⊕ 1 = 0

P8 covers the positions  9-15:
So we get P8 = 0 ⊕ 1 ⊕ 0 ⊕ 1 = 0

So, we calculated, that our data with Hamming code should look like this : 000010001001

Note : These calculations are based on the assumption, that we want the sum of our bits to be even. The same algorithm will be used, if we want the sum of our bits to be odd. But both the sender and the receiver should be aware which variant is chosen.

## Decoding and checking for errors. Syndrome vectors.

Let us assume, that the receiver has received the code word : 000010001001. We start decoding the way back :

P1 is the result of the XOR sum of positions 3, 5, 7, 9, 11 - 0 ⊕ 1 ⊕ 0 ⊕ 1 ⊕ 0 = 0 (Expected: 0, received: 0, so no error in P1)
P2 is P2 is the result of the XOR sum of positions 3, 6, 7, 10,0 ⊕ 0 ⊕ 0 ⊕ 0 ⊕ 0 = 0 (Expected: 0, received: 0, so no error in P2) 0 = 1
P4 is the result of the XOR sum of posins 4, 5, 6, 7,1120  ⊕ 0 ⊕ 10 ()Expectrded: 0, received: 0, so no errors in P4

P8 is the result of the XOR sum of positions 9, 10, 11, 12 - 1 ⊕ 0 ⊕ 0 ⊕ 1 = 0 (Expected: 0, received: 0, so no errors in P8).

This is gfine. But let us assume, that there was noise during the data transition, and actually the 10th bit received was 1, and not zero: 
000010001101

Then when calculating the parity bits, we would get for P2 and P2 1 and not zeros.

### What is a syndrome vector?
The syndrome vector is analyzed to identify the presence and location of errors in the received codeword. Each entry in the syndrome vector corresponds to a parity-check equation. If the syndrome vector is non-zero, it indicates that an error has occurred. Each parity bit is recalculated using the received codeword and compared with the corresponding parity bit in the received codeword. The syndrome bits form a vector.

Let's recalculate the parity bits in the case, when there was an error in the data transition:

P1 is the result of the XOR sum of positions 3, 5, 7, 9, 11 - 0 ⊕ 1 ⊕ 0 ⊕ 1 ⊕ 0 = 0 (Expected: 0, received: 0, so no error in P1)
P2 is the result of the XOR sum of positions 3, 6, 7, 10, 11 - 0 ⊕ 0 ⊕ 0 ⊕ 1 ⊕ 0 = 1 (Expected: 0, received: 1, so error detected in P2)
P4 is the result of the XOR sum of positions 5, 6, 7, 12 - 1 ⊕ 0 ⊕ 0 ⊕ 1 = 0 (Expected: 0, received: 0, so no errors in P4)
P8 is the result of the XOR sum of positions 9, 10, 11, 12 - 1 ⊕ 1 ⊕ 0 ⊕ 1 = 1 (Expected: 0, received: 1, so eror detected in P8).

Let's now calculate the XOR of the parity bits at the sender and the receiver.
P1 at sender is 0, P1 at receiver is 0, so there is no difference between them. Respectfully the syndrome byt is 0.
P2 at sender is 0, P2 at receiver is 1, so there is difference between them. Respectfully the syndrome byt is 1.
P4 at sender is 0, P4 at receiver is 0, so there is no difference between them. Respectfully the syndrome byt is 0.
P8 at sender is 0, P8 at receiver is 1, so there is difference between them. Respectfully the syndrome byt is 1.

Thus the syndrome vector would look like : (1, 0, 1, 0). But this is exactly the binary representation of 10 - the bit, where the error occurs.

## Implementation

In [4]:
def calculate_parity_bits(data_bits):
    """
    Calculate the number of parity bits required for the given data bits.
    """
    m = len(data_bits)
    for r in range(m):
        if 2**r >= m + r + 1:
            return r

def encode_hamming(data_bits):
    """
    Encodes the given data bits using Hamming code and returns the encoded bits.
    """
    m = len(data_bits)
    r = calculate_parity_bits(data_bits)
    n = m + r

    # Initialize the encoded bits with 0s
    encoded_bits = ['0'] * n

    # Position the data bits in the encoded bits array (1-based index for parity calculation)
    j = 0
    for i in range(1, n + 1):
        if i & (i - 1) == 0:  # Check if the position is a power of 2 (parity bit position)
            encoded_bits[i - 1] = '0'  # Temporary parity bit (will calculate later)
        else:
            encoded_bits[i - 1] = data_bits[j]
            j += 1

    # Calculate parity bits
    for i in range(r):
        parity_pos = 2**i
        parity = 0
        for j in range(1, n + 1):
            if j & parity_pos:
                parity ^= int(encoded_bits[j - 1])
        encoded_bits[parity_pos - 1] = str(parity)

    return ''.join(encoded_bits)

In [5]:

result = encode_hamming('10101010')
print(result)

111101001010


In [8]:
def decode_hamming(encoded_bits):
    """
    Decodes the given Hamming code word and returns the corrected data bits if there's a single-bit error.
    """
    n = len(encoded_bits)
    r = calculate_parity_bits(encoded_bits)
    m = n - r

    # Calculate syndrome
    syndrome = 0
    for i in range(r):
        parity_pos = 2**i
        parity = 0
        for j in range(1, n + 1):
            if j & parity_pos:
                parity ^= int(encoded_bits[j - 1])
        if parity != 0:
            syndrome += parity_pos

    # Correct the error if syndrome is non-zero
    if syndrome != 0:
        print(f"Error detected at position: {syndrome}")
        encoded_bits = list(encoded_bits)
        encoded_bits[syndrome - 1] = str(1 - int(encoded_bits[syndrome - 1]))  # Flip the bit
        encoded_bits = ''.join(encoded_bits)
    else:
        print("No error detected.")

    # Extract data bits
    data_bits = []
    for i in range(1, n + 1):
        if i & (i - 1) != 0:  # Not a power of 2 (data bit position)
            data_bits.append(encoded_bits[i - 1])

    return ''.join(data_bits)

In [10]:
#Decode the result received after the encoding
result = decode_hamming('111101001010')
print(result)

No error detected.
10101010


In [11]:
#Let change the last bit from 0 to 1
result = decode_hamming('111101001011')
print(result)

Error detected at position: 12
10101010


## Parity check matrix, Generator matrix, Syndrome vector of Hamming(7, 4) code

### Generator matrix

The operation of generating codes from data bits may be executed by multiplying the input data bits with some definite matrix of suitable dimension, called the generator matrix. 

A linear (n, k) code has k information symbols and n-k check symbols. In fact, let K be k-dimentional subspace of a finite field with size n, and let $$ e_{1}, e_{2}, \ldots, e_{n} $$ is the basis K. Then every code word has the form :  $$ v = \sum_{i=0}^{k} e_{i}u_{i} $$ for a unique k-tuple of scalars $ u_{i} $. In other words, the k information symbols $$ u_{1}, u_{2}, \ldots, u_{n} $$ determine a code word uniquely by (16).

Let _K_ be linear code with basis $ e_{1}, e_{2}, \ldots, e_{n} $. Then the k x n matrix _G_ with rows $$  \begin{pmatrix}
e_1 \\
e_2 \\
\vdots \\
e_k
\end{pmatrix} $$ is called a generator matrix of the code _K_.

The generator matrix of the Hamming code(7, 4) is:

$$
G = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{bmatrix}$$

The first 4 columns of this matrix form the identity matrix.[3] - p.116. 

How we get this? As it was described in the general algorithm above (and also in [4]), if d1, d2, d3, d4 are the data bits in the Hamming(7, 4) code, the parity bits are calculated in the following way:

p1 = d1 ⊕ d2 ⊕ d4

p2 = d1 ⊕ d3 ⊕ d4

p3 = d2 ⊕ d3 ⊕ d4

So basically we take these equations and put them in the matrix form:

d1 = d1

d2 = d2

d3 = d3

d4 = d4

p1 = d1 ⊕ d2 ⊕ d4

p2 = d1 ⊕ d3 ⊕ d4

p3 = d2 ⊕ d3 ⊕ d4

So expanding this a little bit we get:

d1 = d1 * 1 ⊕ d2 * 0 ⊕ d3 * 0 ⊕ d4 * 0

d2 = d1 * 0 ⊕ d2 * 1 ⊕ d3 * 0 ⊕ d4 * 0

d3 = d1 * 0 ⊕ d2 * 0 ⊕ d3 * 1 ⊕ d4 * 0

d4 = d1 * 0 ⊕ d2 * 0 ⊕ d3 * 0 ⊕ d4 * 1

p1 = d1 * 1 ⊕ d2 * 1 ⊕ d3 * 0 ⊕ d4 * 1

p2 = d1 * 1 ⊕ d2 * 0 ⊕ d3 * 1 ⊕ d4 * 1

p3 = d1 * 0 ⊕ d2 * 1 ⊕ d3 * 1 ⊕ d4 * 1

The "*" symbol represents multiplication modulo 2.

In matrix form this is: 


$$ 
(d1, d2, d3, d4, p1, p2, p3)
= (d1, d2, d3, d4)
\begin{pmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{pmatrix}
$$

### Parity check matrix

For Hamming(7, 4) the code words x are the 7-bit vectors, which satisfy H * xT = 0.

A binary linear code is called Hamming code provided that it has for some number m a parity check matrix _H_ or m rows and 2m-1 columns, so that each nonzero binary word of length m is a column of _H_.

For m = 3, the parity check matrix has 3 rows and 7 columns. We can choose any the orderings of the columns, but the most convenient is to use the binary expansion of 1, 2.7ix}$$
7.

$$
H = \begin{bmatrix}
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
0 & 1 & 1 & 0 & 0 & 1 & 1 \\
1 & 0 & 1 & 0 & 1 & 0 & 1
\end{bmatrix}$$

The rows of a parity check matrix are the coefficients of the parity check equations. That is, they show how linear combinations of certain digits (components) of each codeword equal zero. That means that the code is determined by the following equations: 
$$                         x_{4} + x_{5} + x_{6} + x_{7} = 0 $$
$$         x_{2} + x_{3}                 + x_{6} + x_{7} = 0 $$
$$ x_{1}         + x_{3}         + x_{5}         + x_{7} = 0 $$

These equations are equal to the following: 
$$ x_{5} = x_{2} + x_{3} + x_{4}$$
$$ x_{6} = x_{1} + x_{3} + x_{4}$$
$$ x_{7} = x_{1} + x_{2} + x_{4}$$

So $ x_{1} ... x_{4}$ can be chosen arbitrarily (these are the information symbols), and the last there symbols are the check symbols. Thus the Hamming code with m = 3 has information rate R=4/7 [3-p.71]

Let us show what the parity check matrix should be, if we have chosen the parity check bits to be at positions P1, P2 and p4 for Hamming Code(7,4):
As described above:
- Parity bit at position 1 covers all bit positions which have the least significant bit set: bit 1 (the parity bit itself), 3, 5, 7
- Parity bit at position 2 covers all bit positions which have the second least significant bit set: bits 2-3, 6-7
- Parity bit at position 4 covers all bit positions which have the third least significant bit set: bits 4-7

Let us name the information bits D1, D2, D3 and D4.
So we want to arrange the parity bits, so that : 

P1 = D1 ⊕ D2 ⊕ D4

P2 = D1 ⊕ D3 ⊕ D4

P3 = D2 ⊕ D3 ⊕ D4



So in this case we shall want the following equation to be executed:

$$ \begin{pmatrix}
P1' \\
P2' \\ 
P4'
\end{pmatrix}
=
\begin{pmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 1 \\
0 & 1 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1
\end{pmatrix}
\begin{pmatrix}
P1 \\
P2 \\
D1 \\
P4 \\
D2 \\
D3 \\
D4
\end{pmatrix} $$

P1', etc, are the parity check values, which should be 0, if there is no error.

## What are the limitations of Hamming codes? How do more advanced error-correcting codes address these limitations?

The Hamming codes are a fundamental class of of error correcting codes. They are effective for detecting and correcting single-bit errors in the transmitted data. But these codes have two main limitations : 
 - 1 Limited error correction and error detection capabilities - the Hamming codes can detect up to two bit errors and can correct one bit error.
 - 2 Redundancy overhead - the Hamming codes introduce additional parity bits. This in case of large data transmission can cause significant overhead.

As the Hamming codes were the "pioneers" among the error correcting codes, many other codes were developed after that. About 10 years later the Reed-Salomon code was developed. It has the possibility to detect and to correct a lot of errors. That is why the Reed-Salomon code is used for example for reading bar codes or QR codes, where the risk of damages is very high. Also Reed-Salomon codes are used in CDs and DVDs, where data can be subject to burst of errors.  


## Research at least one other error-correcting codes used in industry. Compare their complexity and effectiveness with Hamming codes. Provide at least one real-world example (and edge cases, if possible).

Reed-Solomon Codes are a form of non-binary cyclic error-correcting code that were invented in 1960 by Irving S. Reed and Gustave Solomon. These codes are particularly adept at correcting burst errors, where multiple contiguous bits are corrupted. They do so by operating on blocks of data as polynomials over finite fields rather than individual bits or bytes.Reed-Solomon codes are widely used in various digital communication and storage systems, including CDs, DVDs, and QR codes, because of their versatility and high error-correcting capability.

A Reed-Solomon code is specified as RS(n, k) with s-bit symbols, meaning the coder takes k data symbols of s bits each and adds parity symbols to make an n-symbol codeword. For example, RS(255, 223) with 8-bit symbols can correct up to 16 symbol errors in each codeword. Given a dataset

X = [x_0, x_1, ..., x_{n-1}]
the encoded data Y is generated by polynomial interpolation to find a polynomial that passes through the points generated by mapping the data to symbols in a finite 12][12]

Thus, the Reed-Solomon codes are much more effective, but also much more complex than the Hamming codes.
Because of the effectiveness in correcting burst errors, Reed–Solomon coding is a key component of the compact disc. It was the first use of strong error correction coding in a mass-produced consumer product, and DAT and DVD use similar schemes. In the CD, two layers of Reed–Solomon coding separated by a 28-way convolutional interleaver yields a scheme called Cross-Interleaved Reed–Solomon Coding (CIRC). The first element of a CIRC decoder is a relatively weak inner (32,28) Reed–Solomon code, shortened from a (255,251) code with 8-bit symbols. This code can correct up to 2 byte errors per 32-byte block. More importantly, it flags as erasures any uncorrectable blocks, i.e., blocks with more than 2 byte errors. The decoded 28-byte blocks, with erasure indications, are then spread by the deinterleaver to different blocks of the (28,24) outer code. Thanks to the deinterleaving, an erased 28-byte block from the inner code becomes a single erased byte in each of 28 outer code blocks. The outer code easily corrects this, since it can handle up to 4 such erasures per block.

The result is a CIRC that can completely correct error bursts up to 4000 bits, or about 2.5 mm on the disc surface. This code is so strong that most CD playback errors are almost certainly caused by tracking errors that cause the laser to jump track, not by uncorrectable error burs7]

DVDs use a similar scheme, but with much larger blocks, a (208,192) inner code, and a (182,172) outer . []ode.ld.

## References

[1] https://en.wikipedia.org/wiki/Error_correction_code

[2] https://en.wikipedia.org/wiki/Convolutional_code

[3] Foundations of Coding: Theory and Applications of Error-Correcting Codes with an Introduction to Cryptography and Information Theory - Jiri Adamek

[4] https://en.wikipedia.org/wiki/Hamming_code